In [20]:
from train_model import *

In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import sys
sys.path.append('/home/shiqi/code/Project2-sensor-case/model_combination_Argos/utils')
import numpy as np
import torch
torch.autograd.set_detect_anomaly(True)
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR
import argparse
import os
import yaml
from load_dataset import cut_slices, load_dataset
from sklearn.decomposition import PCA
import copy
import torch.nn.functional as F

In [22]:
config_path = '/home/shiqi/code/Project2-sensor-case/model_combination_Argos/combined_model_20240805/outputs/experiment_1/config.yaml'
config = read_config_file(config_path)

In [23]:
## Device
device = 'cpu'
    
## Load data
x_data, y_data, u_data = data_preparation(config, config['train_data_dir'])
x_data = torch.tensor(x_data, dtype=torch.float32).to(device)
y_data = torch.tensor(y_data, dtype=torch.float32).to(device)
u_data = torch.tensor(u_data, dtype=torch.float32).to(device)
dataset = [x_data, y_data, u_data]

## Dimension
x_dim = x_data.shape[-1]
u_dim = u_data.shape[-1]


## PCA
# Standardize data
mean_1 = torch.mean(x_data, dim=0)
std_1 = torch.std(x_data, dim=0)
std_layer_1 = StdScalerLayer(mean_1, std_1)
x_data_scaled = std_layer_1.transform(x_data)

# PCA layer
pca = PCA(n_components=config['pca_dim'])
# Ensure x_data_scaled is converted back to a NumPy array for PCA
pca.fit(x_data_scaled.detach().cpu().numpy())
components = pca.components_
pca_matrix = torch.tensor(components, dtype=torch.float32).to(device)
print(f'PCA matrix shape: {pca_matrix.shape}')
pca_layer = PCALayer(x_dim, config['pca_dim'], pca_matrix)

# Standardize data 2
x_pca = pca_layer.transform(x_data_scaled)
mean_2 = torch.mean(x_pca, dim=0)
std_2 = torch.std(x_pca, dim=0)
std_layer_2 = StdScalerLayer(mean_2, std_2)

# Build pca dataset
x_pca_scaled = std_layer_2.transform(x_pca)
y_data_scaled = std_layer_1.transform(y_data)
y_pca = pca_layer.transform(y_data_scaled)
y_pca_scaled = std_layer_2.transform(y_pca)
mean_u = torch.mean(u_data, dim=0)
std_u = torch.std(u_data, dim=0)
std_layer_u = StdScalerLayer(mean_u, std_u)
u_data_scaled = std_layer_u.transform(u_data)
dataset = [x_pca_scaled, y_pca_scaled, u_data_scaled]
print(f'PCA data shape: {x_pca_scaled.shape}, {y_pca_scaled.shape}, {u_data_scaled.shape}')


window_size: 150
(149, 6957) (149, 6957) (149, 2)
x_data shape: (6258, 6957), y_data shape: (6258, 6957), u_data shape: (6258, 2)
PCA matrix shape: torch.Size([4, 6957])
PCA data shape: torch.Size([6258, 4]), torch.Size([6258, 4]), torch.Size([6258, 2])


/home/shiqi/code/Project2-sensor-case/model_combination_Argos/combined_model_20240805/train_model.py:311: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.mean = nn.Parameter(torch.tensor(mean, dtype=torch.float32), requires_grad=False)
/home/shiqi/code/Project2-sensor-case/model_combination_Argos/combined_model_20240805/train_model.py:312: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.std = nn.Parameter(torch.tensor(std, dtype=torch.float32), requires_grad=False)


In [24]:
linear_model = Linear_model(config['pca_dim']+1, u_dim)
params = Params(x_dim, u_dim, config)
state_dict = State_Encoder(params)
control_dict = Control_Encoder(params)
residual_model = PCAKoopman(params, std_layer_1, pca_layer, std_layer_2, std_layer_u, state_dict, control_dict)

In [25]:
err_data = y_pca_scaled - linear_model(x_pca_scaled, u_data_scaled)

In [26]:
std_layer_err = StdScalerLayer(torch.mean(err_data, dim=0), torch.std(err_data, dim=0))

In [28]:
print(f'Error data shape: {err_data.shape}')

Error data shape: torch.Size([6258, 4])


In [30]:
linear_pred = linear_model(x_pca_scaled, u_data_scaled)
residual_pred = residual_model.latent_to_latent_forward(x_pca_scaled, u_data_scaled, std_layer_err)
y_pred = linear_pred + residual_pred

In [31]:
mse = torch.nn.MSELoss()
loss = mse(y_pca_scaled, y_pred)
print(f'Loss: {loss}')

Loss: 1080.5859375


In [32]:
x_data_slices = cut_slices(x_pca_scaled, config['window_size'] - 1, 2)
y_data_slices = cut_slices(y_pca_scaled, config['window_size'] - 1, 2)
u_data_slices = cut_slices(u_data_scaled, config['window_size'] - 1, 2)

In [33]:
x_data_slices = torch.cat(x_data_slices, dim=0).to(device)
y_data_slices = torch.cat(y_data_slices, dim=0).to(device)
u_data_slices = torch.cat(u_data_slices, dim=0).to(device)

In [34]:
print(f'Slices shape: {x_data_slices.shape}, {y_data_slices.shape}, {u_data_slices.shape}')

Slices shape: torch.Size([6216, 2, 4]), torch.Size([6216, 2, 4]), torch.Size([6216, 2, 2])


In [37]:
print(hybrid_loss(linear_model, residual_model, std_layer_err=std_layer_err, x_data=x_data_slices, u_data=u_data_slices))

tensor(537.6321, grad_fn=<MseLossBackward0>)


In [38]:
linear_pred = linear_model(x_data_slices[:, 0, :], u_data_slices[:, 0, :])
residual_pred = residual_model.latent_to_latent_forward(x_data_slices[:, 0, :], u_data_slices[:, 0, :], std_layer_err)
y_pred = linear_pred + residual_pred
loss1 = mse(x_data_slices[:, 1, :], y_pred)
print(f'Loss1: {loss1}')
loss2 = mse(y_data_slices[:, 0, :], y_pred)
print(f'Loss2: {loss2}')


Loss1: 1086.8543701171875
Loss2: 1086.8543701171875
